# 关心每一层的变化

我发现，MLP和LeNet过滤之后的数据的betti number 表现出的规律性远远超出了data本身的规律性，这是令人难以理解的，有两种方法可以解释：
1. data中，`dataloader`加载数据的时候，没有对每一个图片进行单独的增强，而是所有的图片是同样的增强
2. `LeNet\MLP`这样的网络确实大大的提取了数据的高维信息，所以反映出了更好的规律性

根据目前的信息来看，第一种的概率是很小的，所以我需要在第二种的假设的基础上进行实验，来观察是不是网络在产生作用，

## 首先做一些测试

测试确定我可以正确的得到每一层`forward`的`output`

In [1]:
import torch
import torch.nn as nn

# 钩子函数，在每一层的forward过程中被调用
def hook_function(module, input, output):
    module.layer_outputs = output

# 定义你的神经网络类
class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.layer1 = nn.Linear(10, 20)  # 第一层
        self.layer2 = nn.Linear(20, 30)  # 第二层
        self.layer3 = nn.Linear(30, 1)   # 最后一层

        # 为每一层注册钩子
        self.layer1.register_forward_hook(hook_function)
        self.layer2.register_forward_hook(hook_function)

    def forward(self, x):
        # 通过每一层
        output1 = self.layer1(x)
        output2 = self.layer2(output1)
        output3 = self.layer3(output2)

        return output3

# 创建神经网络对象
net = MyNetwork()

# 定义输入数据
x = torch.randn(1, 10)  # 假设输入数据维度为1x10

# 前向传播，并获取每一层的输出
output = net(x)

# 获取每一层的输出
output1 = net.layer1.layer_outputs
output2 = net.layer2.layer_outputs
output3 = output

print(output1)  # 第一层的输出
print(output2)  # 第二层的输出
print(output3)  # 最后一层的输出

tensor([[ 0.4867, -0.2654, -0.5861,  0.8512,  0.5210,  0.2591,  1.1168,  0.0397,
          0.8878,  0.8383,  0.6029,  0.0833, -0.1011, -0.1284,  0.5959,  0.0568,
          0.9053, -0.8213,  1.2861,  0.5938]], grad_fn=<AddmmBackward0>)
tensor([[ 0.4297,  0.2590, -0.4412, -0.0247,  0.5788, -0.1795, -0.1013,  0.2112,
          0.5665, -0.6485, -0.2779,  0.5817, -0.6824, -0.0286, -0.2177, -0.8587,
          0.2731, -0.3415, -0.3272, -0.1022, -0.7625,  0.0580, -0.3150, -0.1322,
         -0.1135, -0.2101,  0.0320, -0.2436,  0.1177, -0.3311]],
       grad_fn=<AddmmBackward0>)
tensor([[-0.1211]], grad_fn=<AddmmBackward0>)


## 然后调用已经准备好的测试

In [29]:
from dataset.check_betti4net import get_layer_output_betti
from net.custome_net import LeNet, MLP
%load_ext autoreload
%autoreload 2

model_LeNet = LeNet()
get_layer_output_betti(model=model_LeNet, seed=15, save_root="./care_layers_output", debug_size=1000, name="LeNet")

model_LeNet = MLP()
get_layer_output_betti(model=model_LeNet, seed=15, save_root="./care_layers_output", debug_size=1000, name="MLP")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
这里是得到距离矩阵，是否使用cuda=True
这里是得到距离矩阵，是否使用cuda=True
./care_layers_output/LeNet/0/
这里是得到距离矩阵，是否使用cuda=True
这里是得到距离矩阵，是否使用cuda=True
./care_layers_output/LeNet/1/
这里是得到距离矩阵，是否使用cuda=True
这里是得到距离矩阵，是否使用cuda=True
./care_layers_output/LeNet/2/
这里是得到距离矩阵，是否使用cuda=True
这里是得到距离矩阵，是否使用cuda=True
./care_layers_output/LeNet/3/
这里是得到距离矩阵，是否使用cuda=True
这里是得到距离矩阵，是否使用cuda=True
./care_layers_output/LeNet/4/


{'LeNet-birth-death-l1_distance': [array([[   0.     ,  960.89386, 1074.8125 , ..., 1193.0536 ,  980.4014 ,
          1154.012  ],
         [ 960.89386,    0.     ,  972.02264, ...,  827.58704,  690.9939 ,
          1166.8242 ],
         [1074.8125 ,  972.02264,    0.     , ...,  995.8995 ,  721.7788 ,
          1284.6969 ],
         ...,
         [1193.0536 ,  827.58704,  995.8995 , ...,    0.     ,  745.2766 ,
          1449.9578 ],
         [ 980.4014 ,  690.9939 ,  721.7788 , ...,  745.2766 ,    0.     ,
          1248.9598 ],
         [1154.012  , 1166.8242 , 1284.6969 , ..., 1449.9578 , 1248.9598 ,
             0.     ]], dtype=float32),
  array([[  0.     , 331.79623, 382.87616, ..., 405.39395, 353.69464,
          418.0963 ],
         [331.79623,   0.     , 246.99892, ..., 267.02734, 140.0346 ,
          457.00464],
         [382.87616, 246.99892,   0.     , ..., 315.7141 , 203.61604,
          464.69687],
         ...,
         [405.39395, 267.02734, 315.7141 , ...,   0.     ,